# **Requirements**

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# **Transfer Learning**


 

---



Importing the libraries

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.inception_v3  import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Image Resize

In [ ]:
IMAGE_SIZE = [224, 224]

Import the dataset

In [ ]:
train_path = '/content/drive/MyDrive/BrainTumor/Train'
test_path = '/content/drive/MyDrive/BrainTumor/Test'

Import the Vgg 16 library and adding preprocessing layer to the front of VGG 

In [ ]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/BrainTumor/Train/*')
folders

['/content/drive/MyDrive/BrainTumor/Train/meningioma_tumor',
 '/content/drive/MyDrive/BrainTumor/Train/glioma_tumor',
 '/content/drive/MyDrive/BrainTumor/Train/no_tumor',
 '/content/drive/MyDrive/BrainTumor/Train/pituitary_tumor']

In [ ]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

Model creation

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
____________________________________________________________________________________________

Choose the best parameters

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Use the Image Data Generator to import the images from the dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/BrainTumor/Train',
                                                 target_size = (224,224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 2870 images belonging to 4 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/BrainTumor/Test',
                                            target_size = (224,224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 394 images belonging to 4 classes.


**fit the model**

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
180/180 [==============================] - 345s 2s/step - loss: 2.9351 - accuracy: 0.7230 - val_loss: 4.0604 - val_accuracy: 0.6574
Epoch 2/20
180/180 [==============================] - 341s 2s/step - loss: 1.7575 - accuracy: 0.8101 - val_loss: 4.2067 - val_accuracy: 0.6827
Epoch 3/20
180/180 [==============================] - 342s 2s/step - loss: 1.6767 - accuracy: 0.8376 - val_loss: 5.1029 - val_accuracy: 0.6954
Epoch 4/20
180/180 [==============================] - 348s 2s/step - loss: 1.2936 - accuracy: 0.8697 - val_loss: 4.4452 - val_accuracy: 0.7640
Epoch 5/20
180/180 [==============================] - 347s 2s/step - loss: 1.6771 - accuracy: 0.8578 - val_loss: 7.8870 - val_accuracy: 0.7335
Epoch 6/20
180/180 [==============================] - 348s 2s/step - loss: 1.0963 - accuracy: 0.9045 - val_loss: 6.8193 - val_accuracy: 0.7107
Epoch 7/20
180/180 [==============================] - 347s 2s/step - loss: 1.1346 - accuracy: 0.8927 - val_loss: 6.5217 - val_accuracy: 0.7741

**Save it as a h5 file**

In [ ]:
from tensorflow.keras.models import load_model

model.save('model_TL_BrainTumor.h5')

Apply model on test set

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
y_pred

array([[0.00000000e+00, 4.53819530e-06, 6.51803184e-29, 9.99995470e-01],
       [0.00000000e+00, 3.68370093e-28, 1.00000000e+00, 8.22110383e-23],
       [1.30654317e-20, 1.00000000e+00, 3.04899966e-25, 1.15621356e-35],
       ...,
       [3.00764555e-06, 9.99996781e-01, 1.89788537e-07, 5.87147939e-25],
       [2.22256747e-17, 1.61799079e-24, 1.00000000e+00, 1.69947429e-23],
       [1.03736351e-22, 1.00000000e+00, 1.07700975e-37, 6.47082261e-20]],
      dtype=float32)

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

array([3, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 0, 2, 2, 1, 1, 2, 0, 1, 1,
       1, 2, 2, 2, 2, 1, 2, 1, 2, 3, 1, 2, 0, 1, 1, 2, 2, 2, 1, 2, 1, 2,
       1, 1, 0, 2, 1, 2, 3, 3, 2, 0, 2, 1, 3, 1, 1, 1, 2, 3, 2, 1, 2, 2,
       1, 1, 1, 2, 2, 2, 2, 1, 0, 1, 1, 2, 2, 1, 0, 1, 3, 2, 2, 1, 1, 1,
       1, 0, 2, 2, 1, 2, 2, 1, 0, 0, 2, 2, 2, 2, 3, 0, 2, 2, 1, 2, 1, 2,
       1, 1, 0, 2, 1, 1, 1, 3, 2, 0, 1, 2, 1, 1, 3, 1, 0, 1, 1, 1, 1, 2,
       2, 1, 2, 2, 1, 0, 3, 1, 1, 1, 2, 1, 2, 0, 3, 3, 0, 2, 2, 1, 3, 2,
       1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 0, 2, 1, 1, 2, 1, 2, 3, 2, 0,
       1, 1, 1, 3, 0, 2, 2, 1, 2, 2, 3, 2, 2, 2, 1, 2, 2, 1, 2, 2, 0, 2,
       3, 2, 2, 2, 2, 3, 1, 0, 2, 2, 2, 2, 1, 1, 3, 1, 2, 2, 1, 1, 2, 2,
       3, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 3,
       2, 1, 1, 1, 1, 2, 1, 0, 1, 2, 0, 3, 3, 1, 1, 1, 2, 3, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 1, 1, 3, 2, 2, 1, 1, 2, 0, 2, 1,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2,

In [ ]:
model=load_model('model_TL_BrainTumor.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/BrainTumor/Test/glioma_tumor/image(11).jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
print(x)
img_data=preprocess_input(x)
img_data.shape

[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[4.9005421e-19, 1.4145679e-16, 1.0000000e+00, 7.9976609e-19]],
      dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)
a

array([2])

In [ ]:
if(a==0):
    print("glioma_tumor")
elif(a==1):
    print("meningioma_tumor")
elif(a==2):
    print("no_tumor")
else:
    print("pituitary_tumor")